# LSSTCam pointing errors

Craig Lage - 16-Apr-25

In [ ]:
import os
import numpy as np
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData

In [ ]:
def getTpointFile(filename):
    data = np.loadtxt(filename, skiprows=5, usecols=(0,1,2,3), comments='END')
    df = pd.DataFrame(data, columns = ['trueAzs', 'trueEls', 'azs', 'els'])
    df['deltaAzs'] = (df['azs'] - df['trueAzs']) * 3600
    df['deltaEls'] = (df['els'] - df['trueEls']) * 3600
    #df['deltaAzs'] = (df['trueAzs'] - df['azs']) * 3600
    #df['deltaEls'] = (df['trueEls'] - df['els']) * 3600
    azs = df['azs'].values
    trueAzs = df['trueAzs'].values
    els = df['els'].values
    trueEls = df['trueEls'].values
    deltaAzs = df['deltaAzs'].values
    deltaEls = df['deltaEls'].values
    return azs, els, trueAzs, trueEls, deltaAzs, deltaEls

In [ ]:
filename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Input_20250528_1.dat"
azs, els, trueAzs, trueEls, deltaAzs, deltaEls = getTpointFile(filename)

plt.figure(figsize=(12,8))
plt.subplots_adjust(hspace=0.3, wspace=0.3)
plt.subplot(1,2,1, aspect=1)
plt.scatter(azs, (np.array(trueAzs) + np.array(deltaAzs) / 3600.0))
plt.subplot(1,2,2, aspect=1)
plt.scatter(els, (np.array(trueEls) + np.array(deltaEls) / 3600.0))


In [ ]:
from scipy.optimize import minimize
def FOM1(params, args):
    fom = 0.0
    [azs, deltaEls] = args
    [amp, off] = params
    for i in range(len(azs)):
        if abs(deltaEls[i]) > 450.0:
            continue
        model = off + amp * np.sin((azs[i]) * np.pi / 180.0)
        err = np.square(model - deltaEls[i])
        fom += err
    return fom

args = [azs, deltaEls]
x0 = [400.0, 0.0]
result1 = minimize(FOM1, x0, args=args, method='Powell')
result1

In [ ]:
from scipy.optimize import minimize
def FOM2(params, args):
    fom = 0.0
    [azs, deltaAzs] = args
    [amp, off] = params
    for i in range(len(azs)):
        if abs(deltaAzs[i]) > 450.0:
            continue
        model = off + amp * np.cos((azs[i]) * np.pi / 180.0)
        err = np.square(model - deltaAzs[i] * np.cos((els[i]) * np.pi / 180.0))
        fom += err
    return fom

args2 = [azs, deltaAzs]
x0 = [400.0, 100.0]
result2 = minimize(FOM2, x0, args=args2, method='Powell')
result2

In [ ]:
def errs1(amp1, off1, azs, deltaEls):
    errs = []
    for i in range(len(azs)):
        model = off1 + amp1 * np.sin((azs[i]) * np.pi / 180.0)
        err = abs(model - deltaEls[i])
        errs.append(err)
    return errs

def errs2(amp2, off2, azs, deltaAzs):
    errs = []
    for i in range(len(azs)):
        model = off2 + amp2 * np.cos((azs[i]) * np.pi / 180.0)
        err = abs(model - deltaAzs[i])
        errs.append(err)
    return errs


In [ ]:
[amp1, off1] = result1.x
[amp2, off2] = result2.x
elErrs = errs1(amp1, off1, azs, deltaEls)
azErrs = errs2(amp2, off2, azs, deltaAzs)

In [ ]:
plt.subplot(1,2,1)
plt.hist(elErrs, bins = 50)
#plt.xlim(0, 5000)
plt.subplot(1,2,2)
plt.hist(azErrs, bins = 50)
#plt.xlim(0, 5000)

In [ ]:
def trim(azs, els, deltaAzs, deltaEls, trueAzs, trueEls, rots, times, dayObss, off1, amp1, off2, amp2, limit=100):
    elErrs = errs1(amp1, off1, azs, deltaEls)
    azErrs = errs2(amp2, off2, azs, deltaAzs)
    newAzs = []; newEls = []; newDeltaAzs = []; newDeltaEls = []; newRots = []
    newTrueAzs = []; newTrueEls = []; newTimes = []; newDayObss = []
    for i in range(len(azs)):
        #if (azErrs[i] < limit) and (elErrs[i] < limit):
        if (elErrs[i] < limit):
            newAzs.append(azs[i]); newEls.append(els[i]); newRots.append(rots[i])
            newDeltaAzs.append(deltaAzs[i]); newDeltaEls.append(deltaEls[i])
            newTrueAzs.append(trueAzs[i]); newTrueEls.append(trueEls[i]); newTimes.append(times[i])
            newDayObss.append(dayObss[i]
    print(len(newEls), len(newDeltaEls), len(newAzs), len(newDeltaAzs), len(newTrueAzs), len(newTrueEls))
    return newAzs, newEls, newDeltaAzs, newDeltaEls, newTrueAzs, newTrueEls, newRots, newTimes, newDayObss
        
    

In [ ]:
azs, els, deltaAzs, deltaEls, trueAzs, trueEls, rots, times, dayObss = trim(azs, els, deltaAzs, deltaEls, trueAzs, trueEls, rots, times, dayObss, off1, amp1, off2, amp2, limit=40)

In [ ]:
print(len(trueAzs), len(trueEls), len(els), len(deltaEls), len(azs), len(deltaAzs), len(times), len(dayObss))

In [ ]:
filename = "/home/c/cslage/u/LSSTCam/data/Tpoint_Input_20250528_1.dat"
azs, els, trueAzs, trueEls, deltaAzs, deltaEls = getTpointFile(filename)
args = [azs, deltaEls]
x0 = [400.0, 0.0]
result1 = minimize(FOM1, x0, args=args, method='Powell')
result1

args2 = [azs, deltaAzs]
x0 = [400.0, 100.0]
result2 = minimize(FOM2, x0, args=args2, method='Powell')
result2

[amp1, off1] = result1.x
[amp2, off2] = result2.x
xs = np.linspace(0, 360, 200)
ys1 = off1 + amp1 * np.sin((xs) * np.pi / 180.0)
ys2 = off2 + amp2 * np.cos((xs) * np.pi / 180.0)
deltaAzsCosEl = np.array(deltaAzs) * np.cos(np.array(els) * np.pi / 180.0)

plt.figure(figsize=(8,8))
plt.suptitle("LSSTCam Delta AltAz 20250527-20250608")
plt.subplots_adjust(hspace=0.3, wspace=0.7)
plt.subplot(2,2,1)
p1 = plt.scatter(els, deltaEls, c=azs, cmap=plt.cm.coolwarm)
cb1 = plt.colorbar(p1)
cb1.set_label('Az')
plt.xlabel('El')
plt.xlim(30, 85)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,2)
p2 = plt.scatter(azs, deltaEls,c=els, cmap=plt.cm.coolwarm)
cb2 = plt.colorbar(p2)
cb2.set_label('El')
plt.plot(xs, ys1, ls = '--', color='black')
plt.text(-100,-50,f"deltaEl={off1:.1f}+\n{amp1:.1f}*sin(az)")
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('Delta El arcsec')
plt.subplot(2,2,3)
p3 = plt.scatter(els, deltaAzsCosEl, c=azs, cmap=plt.cm.coolwarm)
cb3 = plt.colorbar(p3)
cb3.set_label('Az')
plt.xlabel('El')
plt.xlim(30, 85)
plt.ylabel('Delta Az*cos(El) arcsec')
plt.subplot(2,2,4)
p4 = plt.scatter(azs, deltaAzsCosEl,c=els, cmap=plt.cm.coolwarm)
plt.plot(xs, ys2, ls = '--', color='black')
plt.text(-100,-50,f"deltaAz={off2:.1f}+\n{amp2:.1f}*cos(az)")
cb4 = plt.colorbar(p4)
cb4.set_label('El')
plt.xlabel('Az')
plt.xlim(0, 360)
plt.ylabel('Delta Az*cos(El) arcsec')
#plt.savefig("/home/c/cslage/u/LSSTCam/data/Delta_AltAz_27May25-08Jun25.png")


In [ ]:
files = [[20250527, 0], [20250528, 0], [20250529, 0], [20250531, 0], [20250601, 0], [20250603, 0], \
    [20250604, 0], [20250527, 1], [20250528, 1], [20250529, 1], [20250530, 1], \
    [20250601, 1], [20250608, 1], [20250527, 2], [20250530, 2], [20250608, 2]]
pdf = PdfPages("/home/c/cslage/u/LSSTCam/data/Delta_AltAz_By_Date.pdf")
fig = plt.figure(figsize=(8,8))
for [dayObs, stateChanges] in files:
    filename = f"/home/c/cslage/u/LSSTCam/data/Tpoint_Input_{dayObs}_{stateChanges}.dat"
    azs, els, trueAzs, trueEls, deltaAzs, deltaEls = getTpointFile(filename)
    args = [azs, deltaEls]
    x0 = [400.0, 0.0]
    result1 = minimize(FOM1, x0, args=args, method='Powell')
    result1
    
    args2 = [azs, deltaAzs]
    x0 = [400.0, 100.0]
    result2 = minimize(FOM2, x0, args=args2, method='Powell')
    result2
    
    [amp1, off1] = result1.x
    [amp2, off2] = result2.x
    xs = np.linspace(0, 360, 200)
    ys1 = off1 + amp1 * np.sin((xs) * np.pi / 180.0)
    ys2 = off2 + amp2 * np.cos((xs) * np.pi / 180.0)
    deltaAzsCosEl = np.array(deltaAzs) * np.cos(np.array(els) * np.pi / 180.0)
    
    plt.suptitle(f"LSSTCam Delta AltAz {dayObs}_{stateChanges}")
    plt.subplots_adjust(hspace=0.3, wspace=0.7)
    plt.subplot(2,2,1)
    p1 = plt.scatter(els, deltaEls, c=azs, cmap=plt.cm.coolwarm)
    cb1 = plt.colorbar(p1)
    cb1.set_label('Az')
    plt.xlabel('El')
    plt.xlim(30, 85)
    plt.ylabel('Delta El arcsec')
    plt.subplot(2,2,2)
    p2 = plt.scatter(azs, deltaEls,c=els, cmap=plt.cm.coolwarm)
    cb2 = plt.colorbar(p2)
    cb2.set_label('El')
    plt.plot(xs, ys1, ls = '--', color='black')
    plt.text(20, 200,f"deltaEl={off1:.1f}+\n{amp1:.1f}*sin(az)")
    plt.xlabel('Az')
    plt.xlim(0, 360)
    plt.ylabel('Delta El arcsec')
    plt.subplot(2,2,3)
    p3 = plt.scatter(els, deltaAzsCosEl, c=azs, cmap=plt.cm.coolwarm)
    cb3 = plt.colorbar(p3)
    cb3.set_label('Az')
    plt.xlabel('El')
    plt.xlim(30, 85)
    plt.ylabel('Delta Az*cos(El) arcsec')
    plt.subplot(2,2,4)
    p4 = plt.scatter(azs, deltaAzsCosEl,c=els, cmap=plt.cm.coolwarm)
    plt.plot(xs, ys2, ls = '--', color='black')
    plt.text(20, 200,f"deltaAz={off2:.1f}+\n{amp2:.1f}*cos(az)")
    cb4 = plt.colorbar(p4)
    cb4.set_label('El')
    plt.xlabel('Az')
    plt.xlim(0, 360)
    plt.ylabel('Delta Az*cos(El) arcsec')
    pdf.savefig(fig)  # saves the current figure into a pdf page
    print(f"Plot for {dayObs}, {stateChanges} finished")
    plt.clf()

pdf.close()
